In [1]:
from typing import Annotated 
from typing_extensions import TypedDict

In [2]:
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper 
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun 

# Arxiv and Wikipedia tools 
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=300)   
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_wrapper)

wikipedia_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=300)
wiki_tool= WikipediaQueryRun(api_wrapper=wikipedia_wrapper)

In [3]:
wiki_tool.invoke("Who won the cricket world cup 2024?")

"Page: 2024 Men's T20 World Cup\nSummary: The 2024 ICC Men's T20 World Cup was the ninth edition of the ICC Men's T20 World Cup, co-hosted by Cricket West Indies and USA Cricket from 1 to 29 June 2024. It was the first major ICC tournament to include matches played in the United States. The West Indie"

In [4]:
arxiv_tool.invoke("Attention is all you need")

"Published: 2024-07-22\nTitle: Attention Is All You Need But You Don't Need All Of It For Inference of Large Language Models\nAuthors: Georgy Tyukin, Gbetondji J-S Dovonon, Jean Kaddour, Pasquale Minervini\nSummary: The inference demand for LLMs has skyrocketed in recent months, and serving\nmodels with "

In [5]:
tools = [wiki_tool]

In [6]:
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]

In [8]:
from langgraph.graph import StateGraph, START, END

In [9]:
graph_builder = StateGraph(State)

In [10]:
from langchain_groq import ChatGroq

In [11]:
import os
llm = ChatGroq(groq_api_key=os.getenv("GROQ_API_KEY"), model="gemma2-9b-it")

In [12]:
llm_with_tools = llm.bind_tools(tools)

In [13]:
def chatbot(state: State):
    return {
        "messages": [llm_with_tools.invoke(state["messages"])]
    }

In [15]:
from langgraph.prebuilt import ToolNode, tools_condition 

In [16]:
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")

tool_node = ToolNode(tools=tools)

graph_builder.add_node("tools", tool_node)


graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge("chatbot", END)

In [18]:
graph = graph_builder.compile()

In [ ]:
user_input = "Hi, I need help with some answers"

events = graph.stream({
    "messages": {
        "role": "user",
        "content": user_input
    }
})

for event in events:
    print(event.values())
    # event["messages"]["content"][-1].pretty_print()

{'chatbot': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'm83adzdwt', 'function': {'arguments': '{"query":"What is the capital of France?"}', 'name': 'wikipedia'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 85, 'prompt_tokens': 987, 'total_tokens': 1072, 'completion_time': 0.154545455, 'prompt_time': 0.042045839, 'queue_time': 0.248960302, 'total_time': 0.196591294}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--d32486ef-baac-466b-a4a4-41eda94062a2-0', tool_calls=[{'name': 'wikipedia', 'args': {'query': 'What is the capital of France?'}, 'id': 'm83adzdwt', 'type': 'tool_call'}], usage_metadata={'input_tokens': 987, 'output_tokens': 85, 'total_tokens': 1072})]}}
{'tools': {'messages': [ToolMessage(content='Page: Closed-ended question\nSummary: A closed-ended question is any question for which a researcher provides research participants wit